In [ ]:
import pandas as pd
import pickle
import re
import numpy as np


In [ ]:
# read in ddd data - pandas doesn't like reading in file without alteration
ddd_summary = pd.read_csv('path_to_ddd_summary_counts.tsv', header=None, sep='\n')
ddd_summary = ddd_summary[0].str.split('\t', expand=True)
ddd_summary = ddd_summary.applymap(lambda x: x.replace('"', '') if (isinstance(x, str)) else x)
ddd_summary = ddd_summary.rename(columns=ddd_summary.iloc[0]).drop(ddd_summary.index[0])
ddd_summary = ddd_summary.applymap(lambda x: x.replace('_', ':') if (isinstance(x, str)) else x)
# remove trailing semicolon
ddd_summary['HPO_i,count_i;'] = ddd_summary.apply(lambda x: x['HPO_i,count_i;'][:-1], axis = 1)

ddd_summary['no_probands'] = [int(i) for i in ddd_summary['no_probands']]


splt_lst = []
for h in ddd_summary['HPO_i,count_i;']:
    k = [item.split(",") for item in h.split(";")]
    l = [(i[0], int(i[1])) for i in k]
    splt_lst.append((l))

ddd_summary['hpo_count_list'] = splt_lst

ddd_summary = ddd_summary.drop(columns=['HPO_i,count_i;'])

In [ ]:
hp_df = pickle.load(open('path_to_hp_annotated_df.p','rb'))
hp_meanfreq_dict = {'HP:0040281':0.895, 'HP:0040282':0.545, 'HP:0040283':0.17, 'HP:0040284':0.025}
hp_df['hp_meanfreq'] = hp_df['Frequency-HPO'].map(hp_meanfreq_dict, na_action='ignore')

In [ ]:
# download from orphadata
or_df = pickle.load(open('orpha_df_genes.p','rb'))

# previously generated disease models
g2p_df_full = pickle.load(open('path_to_g2p_df.p','rb'))

# orphacode to mim mappings
or_mim_map = pickle.load(open('orph_mim_map.p','rb'))
or_mim_map = or_mim_map.astype('int64')

g2p_df = g2p_df_full.merge(or_mim_map,how='left',left_on='disease mim',right_on='OMIM')

In [ ]:
hp_dis = list(set(hp_df['disease-ID for link']))

In [ ]:
hp_models = []
for dis in hp_dis:
    hp_mod_df = hp_df[hp_df['disease-ID for link']== dis]
    hp_mod = list(zip(hp_mod_df['HPO-Term-ID'], hp_mod_df['hp_meanfreq']))
    hp_models.append({'hp_dis':dis, 'hp_mean_model':hp_mod})

hpoa_df = pd.DataFrame(hp_models)

In [ ]:
g2p_mim_or = g2p_df.copy()

hp_mim_df = hpoa_df[hpoa_df['hp_dis'].str.startswith('OMIM')].copy()
hp_orph_df = hpoa_df[hpoa_df['hp_dis'].str.startswith('ORPHA')].copy()

hp_mim_df['mim'] = [re.sub('OMIM:','',i) for i in hp_mim_df['hp_dis']]
hp_mim_df['mim'] = hp_mim_df['mim'].astype('int64')
hp_orph_df['orpha'] = [re.sub('ORPHA:','',i) for i in hp_orph_df['hp_dis']]
hp_orph_df['orpha'] = hp_orph_df['orpha'].astype('int64')

hp_mim_df = hp_mim_df.rename(columns={'hp_mean_model':'hp_mim_mean_model'})
hp_orph_df = hp_orph_df.rename(columns={'hp_mean_model':'hp_orph_mean_model'})

In [ ]:
hp_mim_df['len_model'] = [len(i) for i in hp_mim_df['hp_mim_mean_model']]

In [ ]:
g2p_mim_or = g2p_mim_or.merge(hp_mim_df,how='left',left_on='disease mim',right_on='mim')

In [ ]:
g2p_mim_or = g2p_mim_or.merge(hp_orph_df,how='left',left_on='Orphanet',right_on='orpha')

In [ ]:
g2p_mim_or = g2p_mim_or.drop(columns= ['hp_dis_x','mim', 'hp_dis_y','orpha'])

In [ ]:
g2p_mim_or = g2p_mim_or.merge(ddd_summary, how='left', left_on='gene symbol', right_on='gene')
g2p_mim_or = g2p_mim_or.rename(columns={'no_probands':'ddd_probands','hpo_count_list':'ddd_hpo'})

In [ ]:
pickle.dump(g2p_mim_or,open('g2p_mim_orph.p','wb'))